# OxDNA Crash course
* [general intro to MD simulations](https://www.public.asu.edu/~psulc/myimages/chapter.pdf)
* [oxDNA model intro](https://dna.physics.ox.ac.uk/index.php/DNA_model_introduction)
* [recent howto](https://ieeexplore.ieee.org/abstract/document/10407580)
* [oxView protocols](https://www.nature.com/articles/s41596-022-00688-5)
* [oxDNA / analysis tools documentation](https://lorenzo-rovigatti.github.io/oxDNA/oat/index.html#)
* [This as a Colab](tinyurl.com/oxdna)

## check we have a working installation
* can be also used to imitate CUDA usage if colab complains

# Setup a simulation
* we'll use the following paper for our test case https://pubs.acs.org/doi/10.1021/jacs.8b07180
* we'll compare the [75 degree](https://nanobase.org/structure/62) and the [30 degree](https://nanobase.org/structure/60) tiles


In [ ]:
import os
current_path = os.getcwd()
current_path


In [ ]:
# get the files
%mkdir ./downloads
%cd ./downloads
!wget https://nanobase.org/file/60/structure/30deg.top
!wget https://nanobase.org/file/60/structure/30deg.dat
!wget https://nanobase.org/file/62/structure/75deg.top
!wget https://nanobase.org/file/62/structure/75deg.dat
!wget https://nanobase.org/file/62/structure/traj75_min.dat
!wget https://nanobase.org/file/60/structure/traj30_min.dat
%cd {current_path}

## take a look at the 30 deg structure

In [ ]:
%cd {current_path}
from oxDNA_analysis_tools.UTILS.oxview import from_path
from_path(
    "./downloads/30deg.top",
    "./downloads/30deg.dat"
)

## and also view the 75 deg structure

In [ ]:
%cd {current_path}
from oxDNA_analysis_tools.UTILS.oxview import from_path
from_path(
    "./downloads/75deg.top",
    "./downloads/75deg.dat"
)

## Now let's setup an equilibrium sampling simulation

In [ ]:
# boilerplate
from oxDNA_analysis_tools.UTILS.boilerplate import get_default_input, Simulation, setup_simulation
from pprint import pprint
params = get_default_input()
print("default sampling")
pprint(params)

In [ ]:
# we have a small very sharp system
# so for the test we can use less steps and sample more often
params["steps"] = "1e6"
params["print_conf_interval"] = "1e3"
params["max_io"]=1000

In [ ]:
%cd {current_path}
# setup and run the 75 deg simulation
s75 = Simulation(
    setup_simulation(
        "./downloads/75deg.top",
        "./downloads/75deg.dat",
        "./75deg/",
        params,
        kill_out_dir=False,
    )
)
#s75.run()

In [ ]:
%cd {current_path}
!mv ./downloads/traj75_min.dat ./75deg/trajectory.dat

In [ ]:
# * we can check on it in several ways
# * plotting the energy shows also simulation progress
s75.plot_energy()

In [ ]:
# setup and run the 75 deg simulation
s30 = Simulation(
    setup_simulation(
        "./downloads/30deg.top",
        "./downloads/30deg.dat",
        "./30deg/",
        params,
        kill_out_dir=True,
    )
)
%cd {current_path}
!mv ./downloads/traj30_min.dat ./30deg/trajectory.dat
#s30.run()

In [ ]:
# here let's check the last conf and is alive
print("simulation running:",s30.is_alive())
s30.view_last()

In [ ]:
# or scroll through the existing configurations
s30.view_traj()

In [ ]:
s30.plot_energy()

# So we produced data, how can we analyze it

## compute and display mean stuctures

### for the 75 deg

In [ ]:
%cd {current_path}
%cd ./75deg/
!oat mean -p2 -d devs.json -o mean.dat ./trajectory.dat
from_path("75deg.top",
          "mean.dat",
          "devs.json"
)

### for the 30 deg

In [ ]:
%cd {current_path}
%cd ./30deg/
!oat mean -p2 -d devs.json -o mean.dat ./trajectory.dat
from_path("30deg.top",
          "mean.dat",
          "devs.json"
)

## now let's check the angles
* by looking at the junction, we can define the angle
thrugh using crossover points as the reference points

In [ ]:
%cd {current_path}
%cd ./75deg/
# using the Rye Reader analyse the 1st junction
from oxDNA_analysis_tools.UTILS.RyeReader import get_confs, describe, inbox


p11 = [33,351,350,34]
p12 = [118,495,496,117]
p21 = [180,159,160,179]
p22 = [265,264,76,75]

def angle_between_vectors(v, w):
    # Calculate dot product
    dot_product = np.dot(v, w)
    # Calculate norms
    norm_v = np.linalg.norm(v)
    norm_w = np.linalg.norm(w)
    # Calculate cosine of the angle
    cos_angle = dot_product / (norm_v * norm_w)
    # Calculate the angle in radians and then convert it to degrees
    angle_rad = np.arccos(cos_angle)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

from tqdm.auto import tqdm
import numpy as np

ti,di = describe("75deg.top", "trajectory.dat")

angles75 = []
for id in tqdm(range(di.nconfs)):
  try:
    conf = inbox(
      get_confs(ti,di, id, 1)[0]
    )
    a1 = np.mean(conf.positions[p11],axis = 0)
    a2 = np.mean(conf.positions[p12],axis = 0)
    b1 = np.mean(conf.positions[p21],axis = 0)
    b2 = np.mean(conf.positions[p22],axis = 0)
    a = a1 - a2
    b = b1 - b2
    angles75.append(
        angle_between_vectors(a,b)
    )
  except:
    pass



In [ ]:
%cd {current_path}
# do same thing for the 30
%cd ./30deg/
# using the Rye Reader analyse the 1st junction
from oxDNA_analysis_tools.UTILS.RyeReader import get_confs, describe, inbox


p22 = [50,659,159,160,658]
p21 = [244,111,112,243]
p11 = [46,320,321,45]
p12 = [347,131,132,346]

def angle_between_vectors(v, w):
    # Calculate dot product
    dot_product = np.dot(v, w)
    # Calculate norms
    norm_v = np.linalg.norm(v)
    norm_w = np.linalg.norm(w)
    # Calculate cosine of the angle
    cos_angle = dot_product / (norm_v * norm_w)
    # Calculate the angle in radians and then convert it to degrees
    angle_rad = np.arccos(cos_angle)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

from tqdm.auto import tqdm
import numpy as np

ti,di = describe("30deg.top", "trajectory.dat")

angles30 = []
for id in tqdm(range(di.nconfs)):
  try:
    conf = inbox(
      get_confs(ti,di, id, 1)[0]
    )
    a1 = np.mean(conf.positions[p11],axis = 0)
    a2 = np.mean(conf.positions[p12],axis = 0)
    b1 = np.mean(conf.positions[p21],axis = 0)
    b2 = np.mean(conf.positions[p22],axis = 0)
    a = a1 - a2
    b = b1 - b2
    angles30.append(
        angle_between_vectors(a,b)
    )
  except:
    pass


## plot the obtained distributions

In [ ]:
%cd {current_path}
%cd ./30deg/
import matplotlib.pyplot as plt
import seaborn as sns
sns.histplot(
    angles30, bins=180, label="30"
)
plt.legend()
plt.savefig(
    "./hist1e6.png"
)


In [ ]:
%cd {current_path}
%cd ./75deg/
import matplotlib.pyplot as plt
import seaborn as sns
sns.histplot(
    angles75, bins=180, label="75"
)
plt.legend()
plt.savefig(
    "./hist1e6.png"
)

In [ ]:
#combined out
import numpy as np
sns.kdeplot(
    angles75, label=75,fill=True
)
mean75 = np.mean(angles75)
std75  = np.std(angles75)
plt.plot(
    [mean75, mean75],
    [0,.25],
    c="red"
)
mean30 = np.mean(angles30)
std30  = np.std(angles30)
sns.kdeplot(
    angles30, label=30, fill=True
)
plt.legend()
plt.plot(
    [mean30, mean30],
    [0,.25],
    c="red"
)
print("structure\tmean\tstd")
print(

    f"75 \t{mean75}\t{std75}"
)
print(

    f"30 \t{mean30}\t{std30}"
)

# Convergence and autocorelation
* quick decay of acf indicates good sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def autocorrelation(signal):
    n = len(signal)
    mean = np.mean(signal)
    var = np.var(signal)
    r = np.correlate(signal - mean, signal - mean, mode='full')[-n:]
    result = r/(var*(np.arange(n, 0, -1)))
    return result


# Plot autocorrelation
plt.figure(figsize=(10, 4))

# Compute autocorrelation
acf = autocorrelation(angles75)
plt.plot(acf,label="75")

acf = autocorrelation(angles30)
plt.plot(acf,label="30")

plt.title('Autocorrelation Function')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.legend()
plt.show()


## data convergence can be checked by plotting hist of half the data
* so for 75

In [ ]:
sns.kdeplot(
    angles75[:int(len(angles75)/2)], label="75 1st",fill=True
)
sns.kdeplot(
    angles75[int(len(angles75)/2):], label="75 2nd",fill=True
)
plt.legend()
plt.show()

In [ ]:
sns.kdeplot(
    angles30[:int(len(angles30)/2)], label="30 1st",fill=True
)
sns.kdeplot(
    angles30[int(len(angles30)/2):], label="30 2nd",fill=True
)
plt.legend()
plt.show()

# Last let's have a look at how the junction looks according to the op

In [ ]:
s30.view_traj(op=angles30)